In [ ]:
import pandas as pd
import numpy as np

In [4]:
cod_municipios = pd.read_csv('../dataset/Código dos Municípios')

In [11]:
pd.options.display.float_format = '{:,.2f}'.format

Função que separa o nome da UF do municício


> Um exemplo do padrão é a alteração de `Alta Floresta D'Oeste (RO)` para duas colunas, uma contendo o nome e outra para a UF

In [5]:
def Separa_Nome_UF(df):
    nomes= []
    ufs = []
    i = 0

    for index, row in df.iterrows():
        tmp = row['Município'].split(" (")
        nomes.append(tmp[0])
        ufs.append(tmp[1].strip(")"))
        

    for index, row in df.iterrows():
        df.iat[i, 0] = nomes[i]
        i = i+1
        
    df['Sigla_UF'] = ''

    i = 0
    for index, row in df.iterrows():
        df.iat[i, df.columns.size - 1] = ufs[i]
        i = i+1

In [ ]:
def separa_nome_cod(df):
    cod = []
    i = 0
    
    for index, row in df.iterrows():
        tmp = row['Município'].split(" ")
        cod.append(tmp[0])
        
    for index, row in df.iterrows():
        df.iat[i, 0] = cod[i]
        i = i+1

Essa função padroniza o nome do município e UF para ser separado futuramente

> Exemplo: Um nome de município esta no padrão  `Alta Floresta D'Oeste-RO` e é alterado para `Alta Floresta D'Oeste (RO)`

In [6]:
def transformar_string(df):
    i = 0
    for index, row in df.iterrows():
        df.iat[i, 0] = row['Município'].replace(" - ", " (")
        df.iat[i,0] = df.iat[i,0] + ")"
        i += 1


Essa função trata os dados da PAM, devido ao fato do processo ser o mesmo para todos os anos



Primeiramente separamos o nome da UF e depois adicionamos o codigo do municipio por meio de uma função MERGE, que utiliza o nome do município e a UF como chaves de identificação para atribuição do código do município.

Com isso é excluido os campos de nome e UF para utilizarmos somente o código do município e movimentamos esse código para ele ser a primeira coluna

por fim a função adiciona o texto _Produção de_ e a unidade de medida(_Toneladas_)



In [7]:
def PAM(df):
    Separa_Nome_UF(df)
    df = df.merge(cod_municipios[['Codigo_Municipio', 'Município', 'Sigla_UF']], how='right',left_on=['Município', 'Sigla_UF'], right_on=['Município', 'Sigla_UF'])
    df = df.drop(['Sigla_UF', 'Município'], axis=1)
    df = df[['Codigo_Municipio'] + [col for col in df.columns if col != 'Codigo_Municipio']] 
    for x in df.columns[1:]:
        df = df.rename(columns={x:'Produção de ' + x + '(toneladas)'})
    return df

Função qur trata os dados referentes a tabela do CEMPRE.

primeiro renomea-se as counas referentes ao nome do município e o salário médio mensal e removemos a coluna que traz o nome da UF por extenso

apos isso formatamos o nome do município e separamos o nome da UF

similar a função da PAM adicionamos também a coluna de codigo do município por meio da função MERGE e removemos o nome e UF da tabela 

e por ultimo formatamos o padrão de medida de dinheiro para o brasileiro.

In [ ]:
def CEMPRE(df):
    df = df.rename(columns={'Municípios': 'Município', 'Salário médio\n mensal \n(R$) (2)': 'Salário médio mensal(R$)'})
    df = df.drop(['Unidades da Federação'], axis=1)
    transformar_string(df)
    Separa_Nome_UF(df)
    df = df.merge(cod_municipios[['Codigo_Municipio', 'Município', 'Sigla_UF']], how='right',left_on=['Município', 'Sigla_UF'], right_on=['Município', 'Sigla_UF'])
    df = df.drop(['Sigla_UF', 'Município'], axis=1)
    df = df[['Codigo_Municipio'] + [col for col in df.columns if col != 'Codigo_Municipio']]
    df = df.round(2)
    df.columns = df.columns.str.replace('\n', ' ')
    df.drop('Salários e outras remunerações (1 000 R$)', inplace=True, axis=1)
    # if df['Salário médio mensal(R$)'].dtypes == 'float64':
    #     df['Salário médio mensal(R$)'] = df['Salário médio mensal(R$)'].apply(lambda x: f"{x:,.1f}".replace(",", "X").replace(".", ",").replace("X", "."))
    return df
    

In [ ]:
def Bolsa_Familia(df):
    df['ano'] = df['anomes'].astype(str).str[:4]
    db = df.groupby(['ibge', 'ano']).agg(
    qtd_familias_total_bolsa_familia=('qtd_familias_beneficiarias_bolsa_familia', 'sum'),
    media_qtd_familias_bolsa_familia=('qtd_familias_beneficiarias_bolsa_familia', 'mean'),
    valor_total_repassado_bolsa_familia=('valor_repassado_bolsa_familia', 'sum'),
    media_valor_repassado_bolsa_familia=('valor_repassado_bolsa_familia', 'mean'),
    ).reset_index()
    db = db.drop('ano', axis=1)
    db = db.round(2)
    # db['media_valor_repassado'] = db['media_valor_repassado'].apply(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
    # db['media_qtd_familias'] = db['media_qtd_familias'].apply(lambda x: f"{x:,.2f}")
    return db

In [ ]:
def BPC(df):
    df['ano'] = df['anomes'].astype(str).str[:4]
    db = df.groupby(['ibge', 'ano']).agg(
    num_beneficiados_bpc=('bpc_ben', 'sum'),
    media_beneficiados_bpc=('bpc_ben', 'mean'),
    num_pcd_beneficiados_bpc=('bpc_pcd_ben', 'sum'),
    media_pcd_beneficiados_bpc=('bpc_pcd_ben', 'mean'),
    num_idoso_beneficiados_bpc=('bpc_idoso_ben', 'sum'),
    media_idoso_beneficiados_bpc=('bpc_idoso_ben', 'mean'),
    valor_total_repassado_bpc=('bpc_val', 'sum'),
    media_valor_repassado_bpc=('bpc_val', 'mean'),
    valor_pcd_repassado_bpc=('bpc_pcd_val', 'sum'),
    media_valor_pcd_repassado_bpc=('bpc_pcd_val', 'mean'),
    valor_idoso_repassado_bpc=('bpc_idoso_val', 'sum'),
    media_valor_idoso_repassado_pbc=('bpc_idoso_val', 'mean'),
    ).reset_index()
    db = db.drop('ano', axis=1)
    db = db.round(2)
    # for col in db.columns[1:]:
    #     db[col] = db[col].apply(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
    return db

In [ ]:
def CNES(df, ano):
    cnes = pd.DataFrame()
    remove = df[ano + '/Jan'].count() - df['Município'].count() - 1
    df = df.iloc[:remove]
    separa_nome_cod(df)
    df.replace('-', np.nan, inplace=True)
    df[df.columns] =  df[df.columns].astype(float)
    cnes['cod_municipio'] = df['Município']
    cnes['sum'] = df[[x for x in df.columns if ano in x]].sum(axis=1)
    return cnes